### Word Group In Youtube Sentence

In [39]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [40]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [41]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 1000  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 20
shift = 0.3  # sentence time shift

# func select
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number
sentence_word_count_number = 5   # False for selecting word group according to max word group length


# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")


6


In [42]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/2-Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [43]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
    text = " ".join(text_list)
    
    return text 

In [44]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [45]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [46]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988210,karneleme,5
988211,karnaya,5
988212,dörtlümüzün,5
988213,karnavalınız,5


In [47]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [48]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
995,görev,71055
996,getirdim,70790
997,dinleyin,70632
998,hale,70625


In [49]:
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abluka,blockade
2,absorbe,absorb
3,absürt,absurd
4,açelya,azalea
...,...,...
1773,zebra,zebra
1774,zikzak,zigzag
1775,zombi,zombie
1776,zooloji,zoology


In [50]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,english_word


In [51]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [52]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [53]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

#### Youtube Sentence Analysis

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [ ]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

In [ ]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

In [ ]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio

In [ ]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

In [ ]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

In [ ]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

In [ ]:
df_word_group_time_loc_in_sent_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_sent_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_sent_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_sent_result

In [ ]:
df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)

In [ ]:
df_word_group_time_loc_in_sent_result.to_excel(f"Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Youtube Videoid Text Analysis

In [54]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [55]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [56]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [57]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

,index,video_id,sentence
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...
...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...


In [58]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

,video_id,start_time
0,007uL_OBtPE,6.680
1,00Dl6tNNlZ8,36.803
2,00QJHZSg0UE,0.440
3,00b3ov_NFSI,0.000
4,00fr8yyBQYo,18.000
...,...,...
18361,zzOL9mRRU0c,9.333
18362,zzWQXUKr9OA,1.896
18363,zzXTIIo7EvM,36.932
18364,zzZfnY7Zjf0,0.000


In [59]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

,video_id,end_time
0,007uL_OBtPE,136.640
1,00Dl6tNNlZ8,813.903
2,00QJHZSg0UE,27.460
3,00b3ov_NFSI,42.400
4,00fr8yyBQYo,413.000
...,...,...
18361,zzOL9mRRU0c,533.254
18362,zzWQXUKr9OA,27.969
18363,zzXTIIo7EvM,1591.680
18364,zzZfnY7Zjf0,15.000


In [60]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

,video_id,start_time,end_time
0,007uL_OBtPE,6.680,136.640
1,00Dl6tNNlZ8,36.803,813.903
2,00QJHZSg0UE,0.440,27.460
3,00b3ov_NFSI,0.000,42.400
4,00fr8yyBQYo,18.000,413.000
...,...,...,...
18361,zzOL9mRRU0c,9.333,533.254
18362,zzWQXUKr9OA,1.896,27.969
18363,zzXTIIo7EvM,36.932,1591.680
18364,zzZfnY7Zjf0,0.000,15.000


In [61]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

,index,video_id,sentence,start_time,end_time
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,6.680,136.640
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,36.803,813.903
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,0.440,27.460
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,0.000,42.400
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...,18.000,413.000
...,...,...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,9.333,533.254
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,1.896,27.969
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,36.932,1591.680
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,0.000,15.000


In [62]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [63]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,31.6,"afiyet, kokuyor, kaşık, sizlerde, eridikten, g...","ister, da, olarak, yemek, üç, umarım, iki, zam..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,13.7,"kararların, israf, çıkışında, gözleri, gelmele...","da, hepimiz, durumda, on, umarım, size, istiyo..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,29.0,"birbirinizi, idünyalara, iyoksa, teklifimdeni,...","de, hem, sen, biraz, ne, şu, işin, ben, vakit,..."
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,46.0,"yabancı, maalesef, çift, gittin, barışırsınız,...","da, terk, ne, mu, mi, kadın, bilmiyorum, bu, b..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,1.0,"olympics, everywhere, health, lennons, address...","at, on, new, an, be, i, a, the"
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,16.7,"avrupa, afiyet, vitamini, kokuyor, atalarımız,...","adını, da, kaldı, eder, aldı, kendini, uzak, n..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,47.4,"baştan, cinayete, güven, hanım, kızınız, kalır...","da, diğer, sana, varsa, dışında, mi, bu, biris..."
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,9.7,"evlerin, bahsettikten, duş, göreceksiniz, dayı...","da, oda, şehir, istiyorum, size, yapacağım, bi..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,39.6,"bakabileceğim, setteyken, videolara, geniş, ta...","bir, için, film, küçük, ben, ihtiyacım, bunun,..."


In [64]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,31.6,"afiyet, kokuyor, kaşık, sizlerde, eridikten, g...","ister, da, olarak, yemek, üç, umarım, iki, zam..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,13.7,"kararların, israf, çıkışında, gözleri, gelmele...","da, hepimiz, durumda, on, umarım, size, istiyo..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,29.0,"birbirinizi, idünyalara, iyoksa, teklifimdeni,...","de, hem, sen, biraz, ne, şu, işin, ben, vakit,..."
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,46.0,"yabancı, maalesef, çift, gittin, barışırsınız,...","da, terk, ne, mu, mi, kadın, bilmiyorum, bu, b..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,1.0,"olympics, everywhere, health, lennons, address...","at, on, new, an, be, i, a, the"
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,16.7,"avrupa, afiyet, vitamini, kokuyor, atalarımız,...","adını, da, kaldı, eder, aldı, kendini, uzak, n..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,47.4,"baştan, cinayete, güven, hanım, kızınız, kalır...","da, diğer, sana, varsa, dışında, mi, bu, biris..."
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,9.7,"evlerin, bahsettikten, duş, göreceksiniz, dayı...","da, oda, şehir, istiyorum, size, yapacağım, bi..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,39.6,"bakabileceğim, setteyken, videolara, geniş, ta...","bir, için, film, küçük, ben, ihtiyacım, bunun,..."


In [65]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,31.6,"afiyet, kokuyor, kaşık, sizlerde, eridikten, g...","ister, da, olarak, yemek, üç, umarım, iki, zam..."
1,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,29.0,"birbirinizi, idünyalara, iyoksa, teklifimdeni,...","de, hem, sen, biraz, ne, şu, işin, ben, vakit,..."
2,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,46.0,"yabancı, maalesef, çift, gittin, barışırsınız,...","da, terk, ne, mu, mi, kadın, bilmiyorum, bu, b..."
3,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...,30.1,"allahın, hlas, meali, suresi, adıyla, rahim, r...","ve, i, olan"
4,00rzeUrFM6E,herkese merhaba feyzadan lezzetler aldım buray...,25.2,"hazırladım, sebzeli, rahatlıkla, iyice, larını...","da, umarım, size, herkese, güç, dakika, şöyle,..."
...,...,...,...,...,...
9628,zzBj92_1ITE,denebilir ki habercinin işi hayatın peşine tak...,38.2,"ortasında, can, taner, yaralanırsa, habercinin...","hoş, baba, ya, bir, sonra, tıpkı, ki, gibi, i,..."
9629,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,32.6,"beğenmeyi, yılbaşı, sözbilen, takipçilerimiz, ...","bütün, merak, için, merhaba, çok, ben, eden, i..."
9630,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,47.4,"baştan, cinayete, güven, hanım, kızınız, kalır...","da, diğer, sana, varsa, dışında, mi, bu, biris..."
9631,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,39.6,"bakabileceğim, setteyken, videolara, geniş, ta...","bir, için, film, küçük, ben, ihtiyacım, bunun,..."


In [66]:
if word_group_adjust_select:
    df_adjust_text_ratio = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=True, sent_len_num=sentence_word_count_number)
else:
    df_adjust_text_ratio.loc[:,"search_string"] = df_adjust_text_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))

df_adjust_text_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,video_id,sentence,word_ratio,different_word,intersect_word,search_string
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,31.6,"afiyet, kokuyor, kaşık, sizlerde, eridikten, g...","ister, da, olarak, yemek, üç, umarım, iki, zam...",istiyorum ayrıca şunu da
1,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,29.0,"birbirinizi, idünyalara, iyoksa, teklifimdeni,...","de, hem, sen, biraz, ne, şu, işin, ben, vakit,...",biraz vakit
2,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,46.0,"yabancı, maalesef, çift, gittin, barışırsınız,...","da, terk, ne, mu, mi, kadın, bilmiyorum, bu, b...",sanıyorsun onu ama ne kadar
3,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...,30.1,"allahın, hlas, meali, suresi, adıyla, rahim, r...","ve, i, olan",i
4,00rzeUrFM6E,herkese merhaba feyzadan lezzetler aldım buray...,25.2,"hazırladım, sebzeli, rahatlıkla, iyice, larını...","da, umarım, size, herkese, güç, dakika, şöyle,...",çok kolay çok da kalın mutlu kalın hoşça kalın...
...,...,...,...,...,...,...
9628,zzBj92_1ITE,denebilir ki habercinin işi hayatın peşine tak...,38.2,"ortasında, can, taner, yaralanırsa, habercinin...","hoş, baba, ya, bir, sonra, tıpkı, ki, gibi, i,...",hoş ama ya hayat
9629,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,32.6,"beğenmeyi, yılbaşı, sözbilen, takipçilerimiz, ...","bütün, merak, için, merhaba, çok, ben, eden, i...",ve çok daha
9630,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,47.4,"baştan, cinayete, güven, hanım, kızınız, kalır...","da, diğer, sana, varsa, dışında, mi, bu, biris...",olan birisi varsa o da
9631,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,39.6,"bakabileceğim, setteyken, videolara, geniş, ta...","bir, için, film, küçük, ben, ihtiyacım, bunun,...",işler yapıyorum ve bunun için


In [67]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

,search_string,video_id,sentence
0,istiyorum ayrıca şunu da,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...
1,biraz vakit,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...
2,sanıyorsun onu ama ne kadar,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
3,i,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...
4,çok kolay çok da kalın mutlu kalın hoşça kalın...,00rzeUrFM6E,herkese merhaba feyzadan lezzetler aldım buray...
...,...,...,...
9628,hoş ama ya hayat,zzBj92_1ITE,denebilir ki habercinin işi hayatın peşine tak...
9629,ve çok daha,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...
9630,olan birisi varsa o da,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...
9631,işler yapıyorum ve bunun için,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...


In [68]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [69]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

,search_string,video_id,sentence,start_time,end_time
0,istiyorum ayrıca şunu da,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,6.680,136.640
1,biraz vakit,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,0.440,27.460
2,sanıyorsun onu ama ne kadar,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,0.000,42.400
3,i,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...,0.000,13.701
4,çok kolay çok da kalın mutlu kalın hoşça kalın...,00rzeUrFM6E,herkese merhaba feyzadan lezzetler aldım buray...,0.000,59.370
...,...,...,...,...,...
9628,hoş ama ya hayat,zzBj92_1ITE,denebilir ki habercinin işi hayatın peşine tak...,0.000,25.193
9629,ve çok daha,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,0.000,23.600
9630,olan birisi varsa o da,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,1.896,27.969
9631,işler yapıyorum ve bunun için,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,0.000,15.000


In [70]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,istiyorum ayrıca şunu da,16.044612,18.749944,merhaba arkadaşlar bugün sizlere karadeniz yör...,007uL_OBtPE
1,biraz vakit,1.095030,2.159455,ioğluyla biraz vakit geçirsini ievlat bui idün...,00QJHZSg0UE
2,sanıyorsun onu ama ne kadar,4.260287,6.221372,evleniyorsun kocam diyorsun aynı yastığa baş k...,00b3ov_NFSI
3,i,0.000000,0.498218,i hlas suresi meali rahman ve rahim olan allah...,00pHXwzua50
4,çok kolay çok da kalın mutlu kalın hoşça kalın...,13.230155,14.683177,herkese merhaba feyzadan lezzetler aldım buray...,00rzeUrFM6E
...,...,...,...,...,...
9628,hoş ama ya hayat,4.905568,6.402182,denebilir ki habercinin işi hayatın peşine tak...,zzBj92_1ITE
9629,ve çok daha,11.225468,12.374532,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,zzNekJQxa3s
9630,olan birisi varsa o da,10.863303,12.671835,bak baştan anlaşalım ya bu cinayete de diğer c...,zzWQXUKr9OA
9631,işler yapıyorum ve bunun için,5.580357,6.964286,ben arjantinli bir yapımcıyım ve taşınabilirli...,zzZfnY7Zjf0


In [71]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,istiyorum ayrıca şunu da,15.744612,19.049944,merhaba arkadaşlar bugün sizlere karadeniz yör...,007uL_OBtPE
1,biraz vakit,0.795030,2.459455,ioğluyla biraz vakit geçirsini ievlat bui idün...,00QJHZSg0UE
2,sanıyorsun onu ama ne kadar,3.960287,6.521372,evleniyorsun kocam diyorsun aynı yastığa baş k...,00b3ov_NFSI
3,i,-0.300000,0.798218,i hlas suresi meali rahman ve rahim olan allah...,00pHXwzua50
4,çok kolay çok da kalın mutlu kalın hoşça kalın...,12.930155,14.983177,herkese merhaba feyzadan lezzetler aldım buray...,00rzeUrFM6E
...,...,...,...,...,...
9628,hoş ama ya hayat,4.605568,6.702182,denebilir ki habercinin işi hayatın peşine tak...,zzBj92_1ITE
9629,ve çok daha,10.925468,12.674532,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,zzNekJQxa3s
9630,olan birisi varsa o da,10.563303,12.971835,bak baştan anlaşalım ya bu cinayete de diğer c...,zzWQXUKr9OA
9631,işler yapıyorum ve bunun için,5.280357,7.264286,ben arjantinli bir yapımcıyım ve taşınabilirli...,zzZfnY7Zjf0


In [72]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,istiyorum ayrıca şunu da,16,19,merhaba arkadaşlar bugün sizlere karadeniz yör...,007uL_OBtPE
1,biraz vakit,1,2,ioğluyla biraz vakit geçirsini ievlat bui idün...,00QJHZSg0UE
2,sanıyorsun onu ama ne kadar,4,7,evleniyorsun kocam diyorsun aynı yastığa baş k...,00b3ov_NFSI
3,i,0,1,i hlas suresi meali rahman ve rahim olan allah...,00pHXwzua50
4,çok kolay çok da kalın mutlu kalın hoşça kalın...,13,15,herkese merhaba feyzadan lezzetler aldım buray...,00rzeUrFM6E
...,...,...,...,...,...
9628,hoş ama ya hayat,5,7,denebilir ki habercinin işi hayatın peşine tak...,zzBj92_1ITE
9629,ve çok daha,11,13,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,zzNekJQxa3s
9630,olan birisi varsa o da,11,13,bak baştan anlaşalım ya bu cinayete de diğer c...,zzWQXUKr9OA
9631,işler yapıyorum ve bunun için,5,7,ben arjantinli bir yapımcıyım ve taşınabilirli...,zzZfnY7Zjf0


In [73]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,yardım et bana baba yardım et baba lütfen yard...,422,482,hem bu da bir işaretti baba gerçekten yani dor...,a4UF4C1hiB0
1,her şey senin yüzünden oldu her şey senin yüzü...,455,513,enver abi günaydın hastaneye mi evet kızım gün...,CdY4NyQpyXM
2,anne anne anne anne anne anne anne anne anne n...,381,420,ay tamam yani evlat evlat da yani sendeki sabı...,mxbyYFlxBc8
3,sessiz ol bırak sessiz ol bırak beni sessiz ol...,313,388,allah belanı versin uygar alçak pislik açın ka...,yB0VyuiSTiw
4,var bu gece sabah olmaz bu gece sabah olmaz bu...,76,92,ölmedim ölemem son günü göreceğim hayalim var ...,de3Kej4JDpw
...,...,...,...,...,...
9628,i,1,1,reklamlariniz i çi n arayin,hn9a8CZPDFo
9629,i,0,3,i nfowood,nRkNExvJQvE
9630,i,12,15,şahi n group i nşaat,0tbFgD5QKGI
9631,i,1,2,i nfowood,tcu9ywC1RUs


In [74]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,yardım et bana baba yardım et baba lütfen yard...,422,482,hem bu da bir işaretti baba gerçekten yani dor...,a4UF4C1hiB0,https://www.youtube.com/watch?v=a4UF4C1hiB0&t=...
1,her şey senin yüzünden oldu her şey senin yüzü...,455,513,enver abi günaydın hastaneye mi evet kızım gün...,CdY4NyQpyXM,https://www.youtube.com/watch?v=CdY4NyQpyXM&t=...
2,anne anne anne anne anne anne anne anne anne n...,381,420,ay tamam yani evlat evlat da yani sendeki sabı...,mxbyYFlxBc8,https://www.youtube.com/watch?v=mxbyYFlxBc8&t=...
3,sessiz ol bırak sessiz ol bırak beni sessiz ol...,313,388,allah belanı versin uygar alçak pislik açın ka...,yB0VyuiSTiw,https://www.youtube.com/watch?v=yB0VyuiSTiw&t=...
4,var bu gece sabah olmaz bu gece sabah olmaz bu...,76,92,ölmedim ölemem son günü göreceğim hayalim var ...,de3Kej4JDpw,https://www.youtube.com/watch?v=de3Kej4JDpw&t=76s
...,...,...,...,...,...,...
9628,i,1,1,reklamlariniz i çi n arayin,hn9a8CZPDFo,https://www.youtube.com/watch?v=hn9a8CZPDFo&t=1s
9629,i,0,3,i nfowood,nRkNExvJQvE,https://www.youtube.com/watch?v=nRkNExvJQvE&t=0s
9630,i,12,15,şahi n group i nşaat,0tbFgD5QKGI,https://www.youtube.com/watch?v=0tbFgD5QKGI&t=12s
9631,i,1,2,i nfowood,tcu9ywC1RUs,https://www.youtube.com/watch?v=tcu9ywC1RUs&t=1s


In [75]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [76]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"Word_Group_Max_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

In [ ]:
text = df_word_group_time_loc_in_text_result.loc[13,"sentence"]
text

In [ ]:
words = re.findall(r"\w+", text, re.UNICODE)
words

In [ ]:
index_list = []
for i in range(len(words)):
    if words[i] in word_list:
        index_list.append(i)
    else:
        pass

In [ ]:
all_index_list = []
var_index_list = []
for j in range(len(index_list)):
    try:
        var1 = index_list[j] + 1  
        var2 = index_list[j+1]
    except:
        var1 = index_list[j] + 1  
        var2 = index_list[-1]
    if var1 == var2:
        var3 = index_list[j]
        var_index_list.append(var3)
    else:
        var3 = index_list[j]
        var_index_list.append(var3)
        var4 = list(var_index_list)
        all_index_list.append(var4)
        var_index_list = []

In [ ]:
all_index_list

In [ ]:
sorted(all_index_list, key=len, reverse=True)

In [ ]:
max(all_index_list, key=len)

In [ ]:
text_list = []
for i in max(all_index_list, key=len):
    word = words[i]
    text_list.append(word)
text = " ".join(text_list)
text

In [ ]:
df_adjust_text_ratio

In [ ]:
df_test = df_adjust_text_ratio.iloc[4:5,:]
#df_test.reset_index(drop=True, inplace=True)
df_test

In [ ]:
# tested
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [ ]:
df_new = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=True, sent_len_num=10)
df_new

In [ ]:
df_new.drop_duplicates()

In [ ]:
#text = df_videoid_sentence.loc[0,"sentence"]
#text

In [ ]:
text = "ablan burada nerede i çeriye kadar gelemedi ama benimle buraya kadar geldi sen şimdi git baba sonra beni almaya gel ama hemen gel tamam mı tamam defol defol defol her şeyi mahvettin defol ne işin var burada hayır hayır hayır bırakın bırak beni bırak beni bırak"

In [ ]:
words = re.findall(r"\w+", text, re.UNICODE)
words

In [ ]:
len(words)

In [ ]:
words[1] in word_list

In [ ]:
index_list = []
for i in range(len(words)):
    if words[i] in word_list:
        index_list.append(i)
    else:
        pass

In [ ]:
index_list

In [ ]:
index_list[0] 

In [ ]:
len(index_list)

In [ ]:
all_index_list = []
var_index_list = []
for j in range(len(index_list)):
    try:
        var1 = index_list[j] + 1  
        var2 = index_list[j+1]
    except:
        var1 = index_list[j] + 1  
        var2 = index_list[-1]
    if var1 == var2:
        var3 = index_list[j]
        var_index_list.append(var3)
    else:
        var3 = index_list[j]
        var_index_list.append(var3)
        var4 = list(var_index_list)
        all_index_list.append(var4)
        var_index_list = []

In [ ]:
all_index_list

In [ ]:
all_index_list[0]

In [ ]:
max(all_index_list, key=len)

In [ ]:
text_list = []
for i in max(all_index_list, key=len):
    word = words[i]
    text_list.append(word)
text = " ".join(text_list)
text